# LAB 2 - Batch AI with Horovod (Multi-GPU, Multi-node)


## Introduction

This recipe shows how to run Horovod distributed training framework using Batch AI.
Currently Batch AI has no native support for Horovod framework, but it's easy to run it using customtoolkit and job preparation command line..

## Details

- Standard Horovod tensorflow_mnist.py example will be used;
- tensorflow_mnist.py downloads training data on its own during execution;
- The job will be run on standard tensorflow container tensorflow/tensorflow:1.1.0-gpu;
- Horovod framework will be installed in the container using job preparation command line. Note, you can build your own docker image containing tensorflow and horovod instead.
- Standard output of the job will be stored on Azure File Share.

## Instructions

### Create in Azure the Resource Groups and Storage Accounts needed.
```
> ssh sshuser@YOUR.VM.IP.ADDRESS
> az login
> az group create --name batchai_rg  --location eastus
> az storage account create --location eastus --name batchaipablo --resource-group batchai_rg --sku Standard_LRS
> az storage account keys list --account-name batchaipablo --resource-group batchai_rg -o table
> az ad sp create-for-rbac --name MyAppSvcPppl --password Passw0rd
> az storage account keys list --account-name batchaipablo --resource-group batchai_rg
```

### Read Configuration and Create Batch AI client

In [1]:
from __future__ import print_function

from datetime import datetime
import os
import sys
import zipfile

from azure.storage.file import FileService
import azure.mgmt.batchai.models as models

# utilities.py contains helper functions
import utilities

# Resource Group
location = 'eastus'
resource_group = 'batchai_rg'

# credentials used for authentication
client_id = 'ec0640c7-61fa-4662-bce4-8a3e931939ac'
secret = 'Passw0rd'
token_uri = 'https://login.microsoftonline.com/72f988bf-86f1-41af-91ab-2d7cd011db47/oauth2/token'
subscription_id = 'b1395605-1fe9-4af4-b3ff-82a4725a3791'

# credentials used for storage
storage_account_name = 'batchaipablo'
storage_account_key = 'y59heteYEbw5nTLBB/b7rj3jUphvs2Iwslg4AsXFSb4G7ZLgJUep4AuccSmST7I3E8Zw4BaUloebK+VyKmGpog=='

# specify the credentials used to remote login your GPU node
admin_user_name = 'sshuser'
admin_user_password = 'Passw0rd.1!!'

In [2]:
from azure.common.credentials import ServicePrincipalCredentials
import azure.mgmt.batchai as batchai
import azure.mgmt.batchai.models as models

creds = ServicePrincipalCredentials(client_id=client_id, secret=secret, token_uri=token_uri)

client = batchai.BatchAIManagementClient(credentials=creds,subscription_id=subscription_id)

### Create File Share

For this example we will create a new File Share with name `batchaisample` under your storage account.

**Note** You don't need to create new file share for every cluster. We are doing this in this sample to simplify resource management for you.

In [3]:
azure_file_share_name = 'batchailab2'
service = FileService(storage_account_name, storage_account_key)
service.create_share(azure_file_share_name, fail_on_exist=False)
print('Done')

Done


### Configure Compute Cluster

- For this example we will use a gpu cluster of STANDARD_NC6 nodes. Number of nodes in the cluster is configured with nodes_count variable;
- We will mount file share at folder with name external. Full path of this folder on a computer node will be $AZ_BATCHAI_MOUNT_ROOT/external;
- We will call the cluster nc6.

So, the cluster will have the following parameters:

In [4]:
azure_file_share = 'external'
nodes_count = 2
cluster_name = 'nc6'
vmsize = "Standard_NC6" 

volumes = models.MountVolumes(
    azure_file_shares=[
        models.AzureFileShareReference(
            account_name=storage_account_name,
            credentials=models.AzureStorageCredentialsInfo(
                account_key=storage_account_key),
            azure_file_url = 'https://{0}.file.core.windows.net/{1}'.format(
                storage_account_name, azure_file_share_name),
            relative_mount_path=azure_file_share)
    ]
)

parameters = models.ClusterCreateParameters(
    location=location,
    vm_size=vmsize,
    virtual_machine_configuration=models.VirtualMachineConfiguration(
        image_reference=models.ImageReference(
            publisher="microsoft-ads",
            offer="linux-data-science-vm-ubuntu",
            sku="linuxdsvmubuntu",
            version="latest")),    
    user_account_settings=models.UserAccountSettings(
        admin_user_name=admin_user_name,
        admin_user_password=admin_user_password),
    scale_settings=models.ScaleSettings(
        manual=models.ManualScaleSettings(target_node_count=nodes_count)
    ),
    node_setup=models.NodeSetup(
        mount_volumes=volumes,
    )
)

### Create Compute Cluster

In [5]:
cluster = client.clusters.create(resource_group, cluster_name, parameters).result()

### Monitor Cluster Creation

Monitor the just created cluster. utilities.py contains a helper function to print out detail status of the cluster.

In [6]:
cluster = client.clusters.get(resource_group, cluster_name)
utilities.print_cluster_status(cluster)

Cluster state: AllocationState.resizing Target: 2; Allocated: 0; Idle: 0; Unusable: 0; Running: 0; Preparing: 0; Leaving: 0


### Deploy Sample Script and Configure the Input Directories

- Download original sample script:

In [7]:
sample_script_url = 'https://raw.githubusercontent.com/uber/horovod/v0.9.10/examples/tensorflow_mnist.py'
utilities.download_file(sample_script_url, 'tensorflow_mnist.py')

- Create a folder in the file share and upload the sample script to it.

In [8]:
samples_dir = "horovod_samples"
service = FileService(storage_account_name, storage_account_key)
service.create_directory(
    azure_file_share_name, samples_dir, fail_on_exist=False)
service.create_file_from_path(
    azure_file_share_name, samples_dir, 'tensorflow_mnist.py', 'tensorflow_mnist.py')
print('Done')

Done


- The job needs to know where to find train_mnist.py script (the script will download MNIST dataset on its own). So, we will configure an input directory for the script:

In [9]:
input_directories = [
    models.InputDirectory(
        id='SCRIPTS',
        path='$AZ_BATCHAI_MOUNT_ROOT/{0}/{1}'.format(azure_file_share, samples_dir))
]
output_directories = [
    models.OutputDirectory(
        id='MODEL',
        path_prefix='$AZ_BATCHAI_MOUNT_ROOT/{0}'.format(azure_file_share),
        path_suffix="Models")]

### Configure Output Directories
We will store standard and error output of the job in File Share:

In [10]:
std_output_path_prefix = "$AZ_BATCHAI_MOUNT_ROOT/{0}".format(azure_file_share)

There are multiple ways to create folders and upload files into Azure File Share - you can use [Azure Portal](https://ms.portal.azure.com), [Storage Explorer](http://storageexplorer.com/), [Azure CLI2](/azure-cli-extension) or Azure SDK for your preferable programming language.
In this example we will use Azure SDK for python to copy files into file share.

The job will be able to reference those directories using $AZ_BATCHAI_INPUT_SCRIPTS environment variable

### Configure Job
- Will use configured previously input and output directories;
- We will use custom toolkit job to run tensorflow_mnist.py on multiple nodes (use node_count parameter to specify number of nodes). Note, Batch AI will create a hostfile for the job, it can be found via $AZ_BATCHAI_MPI_HOST_FILE environment variable;
- Horovod framework will be installed by job preparation command line;
- Will output standard output and error streams to file share.

You can delete container_settings from the job definition to run the job directly on host DSVM.

In [11]:
job_name = datetime.utcnow().strftime("horovod_%m_%d_%Y_%H%M%S")
parameters = models.job_create_parameters.JobCreateParameters(
     location=location,
     cluster=models.ResourceId(cluster.id),
     node_count=nodes_count,
     input_directories=input_directories,
     output_directories=output_directories,
     std_out_err_path_prefix=std_output_path_prefix,
     container_settings=models.ContainerSettings(
         models.ImageSourceRegistry(image='tensorflow/tensorflow:1.1.0-gpu')),
     job_preparation=models.JobPreparation(
         command_line="apt update; apt install mpi-default-dev mpi-default-bin -y; pip install horovod"),
     custom_toolkit_settings = models.CustomToolkitSettings(
         command_line='mpirun -mca btl_tcp_if_exclude docker0,lo --allow-run-as-root --hostfile $AZ_BATCHAI_MPI_HOST_FILE python $AZ_BATCHAI_INPUT_SCRIPTS/tensorflow_mnist.py'))

### Create a training Job and wait for Job completion


In [12]:
job = client.jobs.create(resource_group, job_name, parameters).result()
print('Created Job: {}'.format(job_name))

Created Job: horovod_02_13_2018_223056


### Wait for Job to Finish
The job will start running when the cluster will have enought idle nodes. The following code waits for job to start running printing the cluster state. During job run, the code prints current content of stdout.

**Note** Execution may take several minutes to complete.

In [13]:
utilities.wait_for_job_completion(client, resource_group, job_name, cluster_name, 'stdouterr', 'stderr.txt')

Cluster state: AllocationState.steady Target: 2; Allocated: 2; Idle: 1; Unusable: 0; Running: 0; Preparing: 1; Leaving: 0
Job state: queued ExitCode: None
Cluster state: AllocationState.steady Target: 2; Allocated: 2; Idle: 1; Unusable: 0; Running: 0; Preparing: 1; Leaving: 0
Job state: queued ExitCode: None
Cluster state: AllocationState.steady Target: 2; Allocated: 2; Idle: 1; Unusable: 0; Running: 0; Preparing: 1; Leaving: 0
Job state: queued ExitCode: None
Cluster state: AllocationState.steady Target: 2; Allocated: 2; Idle: 1; Unusable: 0; Running: 0; Preparing: 1; Leaving: 0
Job state: queued ExitCode: None
Cluster state: AllocationState.steady Target: 2; Allocated: 2; Idle: 1; Unusable: 0; Running: 0; Preparing: 1; Leaving: 0
Job state: queued ExitCode: None
Cluster state: AllocationState.steady Target: 2; Allocated: 2; Idle: 2; Unusable: 0; Running: 0; Preparing: 0; Leaving: 0
Job state: queued ExitCode: None
Cluster state: AllocationState.steady Target: 2; Allocated: 2; Idle: 2

2018-02-13 22:36:17.110423: I tensorflow/core/common_runtime/gpu/gpu_device.cc:887] Found device 0 with properties: 
name: Tesla K80
major: 3 minor: 7 memoryClockRate (GHz) 0.8235
pciBusID f568:00:00.0
Total memory: 11.17GiB
Free memory: 11.10GiB
2018-02-13 22:36:17.110465: I tensorflow/core/common_runtime/gpu/gpu_device.cc:908] DMA: 1 
2018-02-13 22:36:17.110478: I tensorflow/core/common_runtime/gpu/gpu_device.cc:918] 1:   Y 
2018-02-13 22:36:17.110497: I tensorflow/core/common_runtime/gpu/gpu_device.cc:977] Creating TensorFlow device (/gpu:0) -> (device: 1, name: Tesla K80, pci bus id: f568:00:00.0)
2018-02-13 22:36:17.112201: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-13 22:36:17.112248: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are availa

### Download stdout.txt and stderr.txt files for the Job

In [14]:
files = client.jobs.list_output_files(resource_group, job_name, models.JobsListOutputFilesOptions("stdOuterr")) 
for f in list(files):
    utilities.download_file(f.download_url, f.name)
print("All files downloaded")

All files downloaded


### Delete the Job

In [15]:
_ = client.jobs.delete(resource_group, job_name)

### Delete the Cluster
When you are finished with the sample and don't want to submit any more jobs you can delete the cluster using the following code.

In [16]:
_ = client.clusters.delete(resource_group, cluster_name)

### Delete File Share
When you are finished with the sample and don't want to submit any more jobs you can delete the file share completely with all files using the following code.

In [17]:
service = FileService(storage_account_name, storage_account_key)
service.delete_share(azure_file_share_name)

True